In [150]:
import requests
import json
import pprint

fields = [
    'files.data_category',
    'files.data_format',
    'files.file_id'
    ]

fields = ",".join(fields)

endpt = "https://api.gdc.cancer.gov/cases"

filters = {
    "op": "and",
    "content":[
        {
            'op': 'in',
            'content': {
                'field': 'project.program.name',  # program
                'value': ['TCGA']
            }
        },
        {
            'op': 'in',
            'content': {
                'field': 'project.name',  # project
                # either 'Glioblastoma Multiforme' or 'Brain Lower Grade Glioma'
                'value': ['Glioblastoma Multiforme']
            }
        },
        {
            "op": "in",
            "content":{
                "field": "cases.project.primary_site",  # primary site
                "value": ["Brain"]
            }
        },
        {
            "op": "in",
            "content":{
                "field": "demographic.race",  # race
                "value": ["White"]
            }
        }
    ]
}

params = {
    "filters": filters,
    "fields": fields,
    'sort': 'submitter_id:asc',
    'size': 1000  # should be >= number of hits outputted
    }
js = requests.post(endpt, headers = {"Content-Type": "application/json"}, json = params).json()
print('warnings: ', js['warnings'])
print('number of hits: ', js['data']['pagination']['total'])
# print(json.dumps(js, indent=2))
case_files = {}
for case in js['data']['hits']:
#     pprint.pprint(case)
    vcfs = []
    if any((d['data_format'] == 'VCF' and \
            d['data_category'] == 'Simple Nucleotide Variation') for d in case['files']):
        for d in case['files']:
            if d['data_format'] == 'VCF':
                vcfs.append(d['file_id'])
        case_files[case['id']] = {'vcfs': vcfs}


warnings:  {}
number of hits:  507


Post Request to Download Multiple Files 

In [158]:
import requests
import json
import re

data_endpt = "https://api.gdc.cancer.gov/data"

for case in list(case_files.keys())[:1]:
    ids = case_files[case]['vcfs']
    print(ids)


params = {"ids": ids}

response = requests.post(data_endpt,
                        data = json.dumps(params),
                        headers={
                            "Content-Type": "application/json"
                            })

['ca9b0df8-a440-4c4f-9bad-4c877da13955', '82c0449d-53cf-4e78-8a13-ffb501ead24f', '5d6a6b23-6b16-4938-ac3f-022fa5367a6c', '18d86dda-71e8-495a-a6f5-4a43e66c2e6f', '5909647c-05c4-47da-9bc9-30b77f21047a', 'b7a0f479-7c15-436f-9c2f-088d1d11ab32', 'f81d138e-62ef-4491-b4d0-65c2e07957b9', 'ca262127-69db-4836-b4d3-a5726e80ce77']


In [159]:
response  # need access token

<Response [403]>

In [ ]:
response_head_cd = response.headers["Content-Disposition"]

file_name = re.findall("filename=(.+)", response_head_cd)[0]

with open(file_name, "wb") as output_file:
    output_file.write(response.content)

In [105]:
file_name

'gdc_download_20200411_204637.810914.tar.gz'